## Implementation of Quantum Key Distribution protocol
        
Quantum Key Distribution (QKD) allows Alice and Bob to share a secret cryptographic key securely, even with an eavesdropper, Eve, present. Its security is grounded in quantum mechanics principles like the no-cloning theorem and uncertainty principle.
       
This notebook simulates the **BB84 protocol**, proposed by Charles Bennett and Gilles Brassard in 1984, a foundational QKD method. We’ll use Qiskit to generate a shared key, detect eavesdropping, and encrypt/decrypt messages!

## Overview of the BB84 Protocol
        
BB84 creates a secure key (0s and 1s) between Alice (sender) and Bob (receiver) using quantum communication. It has three phases:
        
### Phase 1 - Sending
Alice generates a random *bitstring* and *bases* (Z: rectilinear, X: diagonal). She encodes bits into qubits:
        
| Bit | Basis | State Encoded |
|:---:|:-----:|:-------------:|
| 0 | Z | $$|0⟩$$ (horizontal) |
| 1 | Z | $$|1⟩$$ (vertical) |
| 0 | X | $$|+⟩$$ (+45°) |
| 1 | X | $$|-⟩$$ (-45°) |
        
Alice sends qubits to Bob via a quantum channel.
        
### Phase 2 - Receiving
Bob measures each qubit in a random basis (Z or X). Matching bases yield Alice’s bit; mismatches give random results (50% chance of 0 or 1).
        
### Phase 3 - Comparing
Alice and Bob publicly compare bases (not bits), keeping bits where bases match (~50%) to form the *sifted key*. They sample bits to compute the Quantum Bit Error Rate (QBER). QBER < 11% suggests a secure channel; higher values indicate Eve. They distill a final secure key.

## Setting Up the Simulation
        
We’ll simulate BB84 with Qiskit’s latest tools:
- `qiskit` for quantum circuits.
- `qiskit_aer` for simulation via `AerSimulator` and `SamplerV2`.
- `numpy` for random bits/bases.
- `matplotlib` for visualizations.
- `binascii` for encryption/decryption.

Run the block below to import libraries.

In [ ]:
!pip install qiskit   #install if needed

In [ ]:
# Block 1 - Import libraries
import numpy as np
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import SamplerV2
import numpy as np
import matplotlib.pyplot as plt
import binascii


print("Libraries imported successfully!")

### Step 1 & 2 - Alice Generates Random Bits and Bases",
        
Alice starts by generating a random bitstring (`alice_bits`) and a list of bases (`alice_bases`), where 0 represents the Z basis (rectilinear) and 1 represents the X basis (diagonal).
        
In the block below, generate 500 random bits and bases for Alice. Why 500? We expect ~50% to survive sifting, and some may be used for QBER checking, leaving a reasonably sized key.

In [ ]:
#BLOCK 2 - Generate Alice's bits and bases
np.random.seed(42) # For reproducibility
n_bits = 500 #Number of bits Alice sends
alice_bits = np.random.randint(2, size=n_bits)
alice_bases = np.random.randint(2, size=n_bits)

print("First 10 bits:", alice_bits[:10])
print("First 10 bases (0=Z, 1=X):", alice_bases[:10])

You should see two lists: `alice_bits` (e.g., [0, 1, 1, ...]) and `alice_bases` (e.g., [0, 1, 1, ...]). Each pair defines a qubit Alice will send.

### Step 3 & 4 - Alice Encodes and Sends Qubits

Alice encodes each bit into a qubit based on the table above. For example:
- Bit 0, Z basis → |0⟩ (no gate)
- Bit 1, X basis → |-⟩ (X gate, then H gate)

She sends these qubits to Bob. In simulation, we’ll encode qubits and pass them to Bob’s measurement step.

Define a function `encode_qubit` to create a quantum circuit for each bit and basis.

In [ ]:
# BLOCK 3 - Encode qubits",
def encode_qubit(bit, basis):
    qc = QuantumCircuit(1, 1)
    if bit == 1:
        qc.x(0)  # |1⟩ for bit=1,
    if basis == 1:
        qc.h(0)  # X basis: |+⟩ or |-⟩,
    return qc

This function prepares one of four states: |0⟩, |1⟩, |+⟩, or |-⟩, simulating photon polarisation.

### Step 5, 6, & 7 - Bob Chooses Bases, Measures, and Decodes
        
Bob generates his own random bases (`bob_bases`) and measures each qubit. If his basis matches Alice’s, he gets her bit; if not, the result is random.

We’ll also simulate Eve’s eavesdropping (optional). If `eve_present=True`, Eve measures in random bases, potentially disrupting the key.

In the block below, define a measurement function and simulate the quantum channel.

In [ ]:
# BLOCK 4 - Bob measures (with optional Eve)\n",
def measure_qubit(qc, basis):
    if basis == 1:
        qc.h(0)  # Measure in X basis,
    qc.measure(0, 0)
    return qc
        
# Simulation parameters
eve_present = False   #True
bob_bases = np.random.randint(2, size=n_bits)
bob_bits = []
simulator = AerSimulator()
sampler = SamplerV2()
     
# Eavesdropper setup
if eve_present:
    eve_bases = np.random.randint(2, size=n_bits)
        
# Quantum channel simulation
for i in range(n_bits):
    # Step 3: Alice encodes
    qc = encode_qubit(alice_bits[i], alice_bases[i])
    # Step 4: Eve intercepts (if present)
    if eve_present:
        qc = measure_qubit(qc, eve_bases[i])
        qc_transpiled = transpile(qc, simulator)
        job = sampler.run([qc_transpiled], shots=1)
        result = job.result()
        eve_bit = int(list(result[0].data['c'].get_counts().keys())[0])
        qc = encode_qubit(eve_bit, eve_bases[i])  # Eve resends
    # Step 6: Bob measures
    qc = measure_qubit(qc, bob_bases[i])
    # Step 7: Bob decodes
    qc_transpiled = transpile(qc, simulator)
    job = sampler.run([qc_transpiled], shots=1)
    result = job. result()
    bob_bit = int(list(result[0].data['c'].get_counts().keys())[0])
    bob_bits.append(bob_bit)        
        
print("First 10 Bob’s bases:", bob_bases[:10])
print("First 10 Bob’s bits:", bob_bits[:10])

Bob’s bits may differ due to basis mismatches or Eve. Let’s sift the keys.

### Step 8 - Sift Bases

Alice and Bob compare bases publicly, retaining bits where bases match (~50%, ~250 bits).

Define `sift_keys` for sifted keys.

In [ ]:
# BLOCK 5 - Sift keys
def sift_keys(alice_bits, alice_bases, bob_bits, bob_bases):
    sifted_key_alice = []
    sifted_key_bob = []
    for i in range(len(alice_bits)):
        if alice_bases[i] == bob_bases[i]:
            sifted_key_alice.append(alice_bits[i])
            sifted_key_bob.append(bob_bits[i])
    return sifted_key_alice, sifted_key_bob

sifted_key_alice, sifted_key_bob = sift_keys(alice_bits, alice_bases, bob_bits, bob_bases)
print("Sifted key length:", len(sifted_key_alice))
print("First 10 sifted bits (Alice):", sifted_key_alice[:10])
print("First 10 sifted bits (Bob):", sifted_key_bob[:10])

Sifted keys should align without Eve. Is she listening?

### Step 9 - Check for Eavesdropping
        
Alice and Bob sample ~25% of sifted bits to compute QBER. QBER < 11% indicates a secure channel; >11% suggests Eve.

Calculate QBER.

In [ ]:
# BLOCK 6 - Calculate QBER\n",
def calculate_qber(key_a, key_b, sample_size):
    if len(key_a) < sample_size:
        sample_size = len(key_a)
    if sample_size == 0:
        return 0
    sample_indices = np.random.choice(len(key_a), sample_size, replace=False)
    errors = sum(key_a[i] != key_b[i] for i in sample_indices)
    return errors / sample_size

sample_size = max(10, len(sifted_key_alice) // 4)
qber = calculate_qber(sifted_key_alice, sifted_key_bob, sample_size)
print(f"QBER: {qber:.2%}")

Expect QBER ~20-30% with Eve, ~0% without.

### Step 10 - Final Key Distillation
        
If QBER < 11%, the sifted key becomes the final key (simplified, no full error correction).

Set the final key.

In [ ]:
# BLOCK 7 - Generate final key
qber_threshold = 0.11
if qber < qber_threshold and len(sifted_key_alice) > 0:
    alice_key = sifted_key_alice
    bob_key = sifted_key_bob
    key_status = "Secure key generated"
else:
    alice_key = []
    bob_key = []
    key_status = "Key discarded due to high QBER"

print(f"Key status: {key_status}")
print(f"Keys match: {alice_key == bob_key}")
if alice_key:
    print(f"Final key length: {len(alice_key)}")
    print(f"First 10 bits: {alice_key[:10]}")
else:
    print("No key generated; cannot compare keys.")

Matching keys with low QBER confirm a secure key!

## Visualizing the Results
        
Visualize sifted keys and basis matches to assess protocol efficiency.

Plot the first 20 sifted bits and basis matches.

In [ ]:
# BLOCK 8 - Visualize results
if len(sifted_key_alice) > 0:
    plt.figure(figsize=(10, 4))
    plt.plot(range(min(20, len(sifted_key_alice))), sifted_key_alice[:20], 'o-', label="Alice's sifted key")
    plt.plot(range(min(20, len(sifted_key_bob))), sifted_key_bob[:20], 'x-', label="Bob's sifted key")
    plt.title("Sifted Key Comparison (First 20 Bits)")
    plt.xlabel("Bit Index")
    plt.ylabel("Bit Value")
    plt.legend()
    plt.grid(True)
    plt.show()

basis_matches = [alice_bases[i] == bob_bases[i] for i in range(n_bits)]
plt.figure(figsize=(10, 4))
plt.plot(range(n_bits), basis_matches, 'o', label="Basis Match (1=Match, 0=Mismatch)")
plt.title("Basis Matching Between Alice and Bob")
plt.xlabel("Bit Index")
plt.ylabel("Match (1 or 0)")
plt.legend()
plt.grid(True)
plt.show()

Key plots show agreement. Basis matches confirm ~50% retention.

### Encrypting and Decrypting Messages

With a secure key, Alice and Bob use a one-time pad for encryption/decryption.

Define helper functions.

In [ ]:
# BLOCK 9 - Encryption and decryption functions
def encrypt_message(unencrypted_string, key):
    if not key or not all(bit in (0, 1) for bit in key):
        raise ValueError("Key must be a non-empty list of 0s and 1s")
    try:
        bits = bin(int(binascii.hexlify(unencrypted_string.encode('utf-8', 'surrogatepass')), 16))[2:]
        bitstring = bits.zfill(8 * ((len(bits) + 7) // 8))
        encrypted_string = ""
        key_length = len(key)
        for i in range(len(bitstring)):
            if bitstring[i] not in ('0', '1'):
                raise ValueError(f"Invalid bit at position {i}: {bitstring[i]}")
            key_bit = key[i % key_length]
            encrypted_bit = int(bitstring[i]) ^ int(key_bit)
            encrypted_string += str(encrypted_bit)
        return encrypted_string
    except UnicodeEncodeError as e:
        raise ValueError(f"Message encoding failed: {e}")

def decrypt_message(encrypted_bits, key):
    if not key or not all(bit in (0, 1) for bit in key):
        raise ValueError("Key must be a non-empty list of 0s and 1s")
    if not encrypted_bits or not all(c in ('0', '1') for c in encrypted_bits):  # Fixed: check encrypted_bits, not key
        raise ValueError("Encrypted bits must be a non-empty string of 0s and 1s")
    try:
        unencrypted_bits = ""
        key_length = len(key)
        for i in range(len(encrypted_bits)):
            key_bit = key[i % key_length]
            unencrypted_bit = int(encrypted_bits[i]) ^ int(key_bit)
            unencrypted_bits += str(unencrypted_bit)
        i = int(unencrypted_bits, 2)
        hex_string = '%x' % i
        n = len(hex_string)
        bits = binascii.unhexlify(hex_string.zfill(n + (n & 1)))
        unencrypted_string = bits.decode('utf-8', 'surrogatepass')
        return unencrypted_string
    except (ValueError, UnicodeDecodeError) as e:
        raise ValueError(f"Decryption failed: {e}")

Encrypt and decrypt a message.

In [ ]:
# BLOCK 10 - Send and receive a message
if alice_key:
    message = "QKD is secure!"
    print("Original Message:", message)
    encrypted_message = encrypt_message(message, alice_key)
    print("Encrypted message:", encrypted_message)
    decrypted_message = decrypt_message(encrypted_message, bob_key)
    print("Decrypted message:", decrypted_message)
else:
    print("No secure key available for encryption.")

## Analysis and Exercises
        
- **Sifted Key Length:** ~250 bits (50% of 500)
- **QBER:** ~0% without Eve, ~20-30% with Eve.
- **Security:** QBER > 11% discards the key.

**Exercises:**
1. Set `eve_present = True` in Block 4. What’s the QBER?
2. Set `n_bits = 1000` in Block 2. How long is the sifted key?
3. Set `sample_size` to `len(sifted_key_alice) // 2` in Block 6. Does QBER change?
4. Plot QBER vs. sample size (10%, 25%, 50%) in a new block.

A matching decrypted message confirms a secure key.

In [ ]:
# BLOCK 11 - Version information
import qiskit
print("Qiskit version:", qiskit.__version__)